In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [13]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [17]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 68s 9ms/step - loss: 1.3312 - acc: 0.4323 - val_loss: 1.1683 - val_acc: 0.4846
Epoch 2/30
7352/7352 [==============================] - 54s 7ms/step - loss: 1.0265 - acc: 0.5618 - val_loss: 0.9187 - val_acc: 0.5945
Epoch 3/30
7352/7352 [==============================] - 80s 11ms/step - loss: 0.8139 - acc: 0.6488 - val_loss: 0.7853 - val_acc: 0.6193
Epoch 4/30
7352/7352 [==============================] - 55s 7ms/step - loss: 0.7096 - acc: 0.6689 - val_loss: 0.7408 - val_acc: 0.6159
Epoch 5/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.6401 - acc: 0.6880 - val_loss: 0.7020 - val_acc: 0.6664
Epoch 6/30
7352/7352 [==============================] - 51s 7ms/step - loss: 0.6230 - acc: 0.6979 - val_loss: 0.7287 - val_acc: 0.7017
Epoch 7/30
7352/7352 [==============================] - 47s 6ms/step - loss: 0.5834 - acc: 0.

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         1        0                   0   
SITTING                  0      419        51        0                   1   
STANDING                 0      132       397        2                   0   
WALKING                  0        0         0      466                  29   
WALKING_DOWNSTAIRS       0        0         0        1                 416   
WALKING_UPSTAIRS         0        1         0       22                  14   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            26  
SITTING                           20  
STANDING                           1  
WALKING                            1  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 434  


In [21]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 589us/step


In [22]:
score

[0.46892408261934193, 0.8965049202578894]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

### LSTM with 64 hidden layer , 0.6 Dropout

In [24]:
epochs = 30
batch_size = 32
n_hidden = 64

In [25]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [27]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 33s 4ms/step - loss: 1.3249 - acc: 0.4149 - val_loss: 1.2714 - val_acc: 0.4177
Epoch 2/30
7352/7352 [==============================] - 31s 4ms/step - loss: 1.1012 - acc: 0.4974 - val_loss: 1.2991 - val_acc: 0.4119
Epoch 3/30
7352/7352 [==============================] - 32s 4ms/step - loss: 1.0031 - acc: 0.5642 - val_loss: 0.9688 - val_acc: 0.6118
Epoch 4/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.9098 - acc: 0.5977 - val_loss: 1.0611 - val_acc: 0.5405
Epoch 5/30
7352/7352 [==============================] - 33s 4ms/step - loss: 0.9908 - acc: 0.5503 - val_loss: 0.7473 - val_acc: 0.6552
Epoch 6/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.7811 - acc: 0.6480 - val_loss: 0.8354 - val_acc: 0.6145
Epoch 7/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.7068 - acc: 0.6763 - val_loss: 0.8029 - val_acc: 0.6573
Epoch 8

In [28]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        25        0                   0   
SITTING                  0      374       114        2                   1   
STANDING                 0       62       468        2                   0   
WALKING                  0        0         0      471                   0   
WALKING_DOWNSTAIRS       0        0         0        0                 417   
WALKING_UPSTAIRS         0        0         1        5                   2   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             2  
SITTING                            0  
STANDING                           0  
WALKING                           25  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 463  


In [29]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 920us/step


In [30]:
score

[0.3560266619481148, 0.9172039362063115]

### LSTM with 128 hidden layer , 0.5 Dropout

In [33]:
epochs = 30
batch_size = 64
n_hidden = 128

In [46]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128)               70656     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [48]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 101s 14ms/step - loss: 1.3629 - acc: 0.4023 - val_loss: 1.2220 - val_acc: 0.5005
Epoch 2/30
7352/7352 [==============================] - 101s 14ms/step - loss: 1.0655 - acc: 0.5375 - val_loss: 1.0993 - val_acc: 0.4795
Epoch 3/30
7352/7352 [==============================] - 96s 13ms/step - loss: 1.0112 - acc: 0.5445 - val_loss: 1.3387 - val_acc: 0.5117
Epoch 4/30
7352/7352 [==============================] - 101s 14ms/step - loss: 0.9601 - acc: 0.5740 - val_loss: 0.8010 - val_acc: 0.6040
Epoch 5/30
7352/7352 [==============================] - 99s 13ms/step - loss: 0.7705 - acc: 0.6198 - val_loss: 0.9493 - val_acc: 0.5833
Epoch 6/30
7352/7352 [==============================] - 99s 13ms/step - loss: 0.7869 - acc: 0.6217 - val_loss: 0.7341 - val_acc: 0.6369
Epoch 7/30
7352/7352 [==============================] - 106s 14ms/step - loss: 0.7029 - acc: 0.6869 - val_loss: 0.7891 - val_acc: 0.6

In [49]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 509        0         0        0                   0   
SITTING                  0      341       132        0                   0   
STANDING                 0       40       492        0                   0   
WALKING                  0        0         0      457                  39   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0       21                   7   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            28  
SITTING                           18  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 443  


In [50]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [51]:
score

[0.38895147200638264, 0.9032914828639295]

### LSTM with 64 hidden layer , 0.6 Dropout

In [53]:
epochs = 30
batch_size = 32
n_hidden = 64

In [54]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [56]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 44s 6ms/step - loss: 1.3844 - acc: 0.3992 - val_loss: 1.2437 - val_acc: 0.5334
Epoch 2/30
7352/7352 [==============================] - 43s 6ms/step - loss: 0.9453 - acc: 0.5687 - val_loss: 0.7735 - val_acc: 0.6532
Epoch 3/30
7352/7352 [==============================] - 43s 6ms/step - loss: 0.7157 - acc: 0.6827 - val_loss: 0.6914 - val_acc: 0.7245
Epoch 4/30
7352/7352 [==============================] - 42s 6ms/step - loss: 0.6581 - acc: 0.7206 - val_loss: 0.6682 - val_acc: 0.7346
Epoch 5/30
7352/7352 [==============================] - 42s 6ms/step - loss: 0.6684 - acc: 0.7592 - val_loss: 0.4976 - val_acc: 0.8229
Epoch 6/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3869 - acc: 0.8719 - val_loss: 0.4056 - val_acc: 0.8473
Epoch 7/30
7352/7352 [==============================] - 48s 7ms/step - loss: 0.2653 - acc: 0.9143 - val_loss: 0.3255 - val_acc: 0.8897
Epoch 8

In [57]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      402        88        0                   0   
STANDING                 0      111       419        0                   1   
WALKING                  0        0         0      454                  24   
WALKING_DOWNSTAIRS       0        0         0        4                 411   
WALKING_UPSTAIRS         0        0         0       10                  12   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           1  
WALKING                           18  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 449  


In [58]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 939us/step


In [59]:
score

[0.2584735866839715, 0.9066847641669494]

### 2 LSTM with 0.6 Dropout

In [78]:
epochs = 30
batch_size = 16
n_hidden_1 = 16
n_hidden_2 = 32

In [79]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True,kernel_initializer='he_normal', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='he_normal'))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 128, 16)           1664      
_________________________________________________________________
dropout_13 (Dropout)         (None, 128, 16)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 32)                6272      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 198       
Total params: 8,134
Trainable params: 8,134
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [81]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 81s 11ms/step - loss: 1.3254 - acc: 0.4917 - val_loss: 1.0015 - val_acc: 0.5891
Epoch 2/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.8637 - acc: 0.6438 - val_loss: 0.8016 - val_acc: 0.6447
Epoch 3/30
7352/7352 [==============================] - 77s 10ms/step - loss: 0.7638 - acc: 0.6499 - val_loss: 0.7822 - val_acc: 0.6464
Epoch 4/30
7352/7352 [==============================] - 77s 11ms/step - loss: 0.7106 - acc: 0.6828 - val_loss: 0.7904 - val_acc: 0.6390
Epoch 5/30
7352/7352 [==============================] - 77s 10ms/step - loss: 0.6412 - acc: 0.7401 - val_loss: 0.6707 - val_acc: 0.7068
Epoch 6/30
7352/7352 [==============================] - 80s 11ms/step - loss: 0.5676 - acc: 0.7825 - val_loss: 0.7105 - val_acc: 0.7129
Epoch 7/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.5116 - acc: 0.8097 - val_loss: 0.6182 - val_acc: 0.7699


In [82]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      395        69        1                   3   
STANDING                 0       95       434        3                   0   
WALKING                  0        0         0      471                  20   
WALKING_DOWNSTAIRS       0        0         0        4                 416   
WALKING_UPSTAIRS         0        0         0       42                  19   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           23  
STANDING                           0  
WALKING                            5  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 410  


In [83]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 1ms/step


In [84]:
score

[0.37058800065051384, 0.9036308109942314]

### 2 LSTM with 0.6 Dropout

In [104]:
epochs = 30
batch_size = 16
n_hidden_1 = 32
n_hidden_2 = 64

In [106]:
from keras.layers.normalization import BatchNormalization

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True,kernel_initializer='he_normal', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
model.add(BatchNormalization())
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='he_normal'))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_24 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 32)           128       
_________________________________________________________________
lstm_26 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 390       
Total params: 30,726
Trainable params: 30,662
Non-trainable params: 64
_________________________________________________________________


In [107]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [108]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 112s 15ms/step - loss: 1.0839 - acc: 0.5547 - val_loss: 0.7810 - val_acc: 0.7095
Epoch 2/30
7352/7352 [==============================] - 106s 14ms/step - loss: 0.6937 - acc: 0.7250 - val_loss: 0.8269 - val_acc: 0.7272
Epoch 3/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.5083 - acc: 0.8234 - val_loss: 0.4994 - val_acc: 0.8310
Epoch 4/30
7352/7352 [==============================] - 117s 16ms/step - loss: 0.3190 - acc: 0.9059 - val_loss: 0.5836 - val_acc: 0.8344
Epoch 5/30
7352/7352 [==============================] - 104s 14ms/step - loss: 0.2739 - acc: 0.9149 - val_loss: 0.4901 - val_acc: 0.8677
Epoch 6/30
7352/7352 [==============================] - 102s 14ms/step - loss: 0.2265 - acc: 0.9308 - val_loss: 0.5953 - val_acc: 0.8032
Epoch 7/30
7352/7352 [==============================] - 102s 14ms/step - loss: 0.2171 - acc: 0.9283 - val_loss: 0.4918 - val_acc: 

In [109]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  1      410        72        0                   4   
STANDING                 0      103       428        1                   0   
WALKING                  0        0         0      454                  39   
WALKING_DOWNSTAIRS       0        0         0        0                 412   
WALKING_UPSTAIRS         0        0         0        1                  21   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            4  
STANDING                           0  
WALKING                            3  
WALKING_DOWNSTAIRS                 8  
WALKING_UPSTAIRS                 449  


In [110]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [111]:
score

[0.475318681690802, 0.9036308109942314]

### 2 LSTM with 0.5 Dropout

In [131]:
epochs = 30
batch_size = 32
n_hidden_1 = 32
n_hidden_2 = 64

In [132]:
from keras.layers.normalization import BatchNormalization

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True,kernel_initializer='glorot_uniform', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(BatchNormalization())
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='glorot_uniform'))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_35 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_34 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 128, 32)           128       
_________________________________________________________________
lstm_36 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_35 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 390       
Total params: 30,726
Trainable params: 30,662
Non-trainable params: 64
_________________________________________________________________


In [133]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [134]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 73s 10ms/step - loss: 0.9050 - acc: 0.6273 - val_loss: 0.6982 - val_acc: 0.7550
Epoch 2/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.4140 - acc: 0.8583 - val_loss: 0.8369 - val_acc: 0.7041
Epoch 3/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.3095 - acc: 0.9057 - val_loss: 0.2999 - val_acc: 0.8870
Epoch 4/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1961 - acc: 0.9340 - val_loss: 0.3071 - val_acc: 0.8996
Epoch 5/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1726 - acc: 0.9387 - val_loss: 0.3658 - val_acc: 0.8789
Epoch 6/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.1729 - acc: 0.9421 - val_loss: 0.9548 - val_acc: 0.7482
Epoch 7/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1844 - acc: 0.9297 - val_loss: 0.2892 - val_acc: 0.8924
Epoch 

In [135]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 521        0         0        0                   0   
SITTING                  3      372       115        0                   0   
STANDING                 0       67       465        0                   0   
WALKING                  0        0         0      465                  14   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0       11                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            16  
SITTING                            1  
STANDING                           0  
WALKING                           17  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 455  


In [136]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [137]:
score

[0.291916581574723, 0.9155072955548015]

### 2 LSTM with 0.6 Dropout

In [155]:
epochs = 30
batch_size = 32
n_hidden_1 = 32
n_hidden_2 = 64

In [156]:
from keras.layers.normalization import BatchNormalization

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True,kernel_initializer='glorot_uniform', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
model.add(BatchNormalization())
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='glorot_uniform'))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_44 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 128, 32)           128       
_________________________________________________________________
lstm_46 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_45 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 390       
Total params: 30,726
Trainable params: 30,662
Non-trainable params: 64
_________________________________________________________________


In [157]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [158]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 84s 11ms/step - loss: 1.0121 - acc: 0.5724 - val_loss: 0.6597 - val_acc: 0.7363
Epoch 2/30
7352/7352 [==============================] - 78s 11ms/step - loss: 0.4939 - acc: 0.8211 - val_loss: 0.3863 - val_acc: 0.8768
Epoch 3/30
7352/7352 [==============================] - 74s 10ms/step - loss: 0.2888 - acc: 0.9087 - val_loss: 0.3332 - val_acc: 0.8870
Epoch 4/30
7352/7352 [==============================] - 72s 10ms/step - loss: 0.3069 - acc: 0.9038 - val_loss: 0.3077 - val_acc: 0.8918
Epoch 5/30
7352/7352 [==============================] - 77s 10ms/step - loss: 0.2132 - acc: 0.9294 - val_loss: 0.2635 - val_acc: 0.9101
Epoch 6/30
7352/7352 [==============================] - 74s 10ms/step - loss: 0.1857 - acc: 0.9309 - val_loss: 0.2308 - val_acc: 0.9087
Epoch 7/30
7352/7352 [==============================] - 74s 10ms/step - loss: 0.1483 - acc: 0.9436 - val_loss: 0.2189 - val_acc: 0.9141


In [159]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  1      376       114        0                   0   
STANDING                 0       73       459        0                   0   
WALKING                  0        0         0      491                   4   
WALKING_DOWNSTAIRS       0        0         0        0                 403   
WALKING_UPSTAIRS         0        0         0        3                   4   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                17  
WALKING_UPSTAIRS                 464  


In [160]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [161]:
score

[0.20900893709525392, 0.9263657957244655]

### 2 LSTM with 0.6 Dropout

In [26]:
epochs = 30
batch_size = 32
n_hidden_1 = 32
n_hidden_2 = 64

In [27]:
from keras.layers.normalization import BatchNormalization

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True,kernel_initializer='glorot_uniform', input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
model.add(BatchNormalization())
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='glorot_uniform'))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 32)           128       
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 30,726
Trainable params: 30,662
Non-trainable params: 64
_________________________________________________________________


In [28]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [29]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 65s 9ms/step - loss: 0.8693 - acc: 0.6500 - val_loss: 0.6769 - val_acc: 0.7424
Epoch 2/30
7352/7352 [==============================] - 61s 8ms/step - loss: 0.3918 - acc: 0.8682 - val_loss: 0.6694 - val_acc: 0.7927
Epoch 3/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.2622 - acc: 0.9153 - val_loss: 0.5943 - val_acc: 0.8548
Epoch 4/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.2173 - acc: 0.9241 - val_loss: 0.3414 - val_acc: 0.9013
Epoch 5/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1808 - acc: 0.9369 - val_loss: 0.3243 - val_acc: 0.8958
Epoch 6/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1719 - acc: 0.9362 - val_loss: 0.3135 - val_acc: 0.9040
Epoch 7/30
7352/7352 [==============================] - 60s 8ms/step - loss: 0.1607 - acc: 0.9425 - val_loss: 0.3262 - val_acc: 0.9209
Epoch 8

In [30]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      398        89        0                   0   
STANDING                 0       85       444        3                   0   
WALKING                  0        2         0      480                  14   
WALKING_DOWNSTAIRS       0        1         0        7                 406   
WALKING_UPSTAIRS         0        6         0       26                  12   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            4  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 6  
WALKING_UPSTAIRS                 427  


In [31]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [32]:
score

[0.38958988227966557, 0.9134713267729895]

### 2 LSTM + 1 CNN with 0.5 Dropout

In [19]:
from keras.layers import Conv1D, MaxPooling1D


In [71]:

epochs = 30
batch_size = 32
n_hidden_1 = 32
n_hidden_2 = 64

In [72]:
#https://blog.goodaudience.com/predicting-physical-activity-based-on-smartphone-sensor-data-using-cnn-lstm-9182dd13b6bc
model = Sequential()
model.add(Conv1D(16, (3), input_shape=(X_train.shape[1],X_train.shape[2]), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2),padding='same'))
model.add(Dropout(0.5))

# Configuring the parameters
model.add(LSTM(n_hidden_1,kernel_initializer='glorot_uniform',return_sequences=True))
# Adding a dropout layer
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='glorot_uniform',return_sequences=False))
# Adding a dropout layer
model.add(Dropout(0.5))

# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 128, 16)           448       
_________________________________________________________________
batch_normalization_19 (Batc (None, 128, 16)           64        
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 64, 16)            0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 64, 16)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 64, 32)            6272      
_________________________________________________________________
lstm_19 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
__________

In [73]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [74]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 87s 12ms/step - loss: 0.9210 - acc: 0.5974 - val_loss: 0.7008 - val_acc: 0.6271
Epoch 2/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.6637 - acc: 0.6604 - val_loss: 0.6607 - val_acc: 0.6284
Epoch 3/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.5996 - acc: 0.7131 - val_loss: 0.6272 - val_acc: 0.6943
Epoch 4/30
7352/7352 [==============================] - 77s 10ms/step - loss: 0.5023 - acc: 0.7888 - val_loss: 0.4980 - val_acc: 0.8120
Epoch 5/30
7352/7352 [==============================] - 75s 10ms/step - loss: 0.3637 - acc: 0.8723 - val_loss: 0.3972 - val_acc: 0.8829
Epoch 6/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.2687 - acc: 0.9074 - val_loss: 0.4035 - val_acc: 0.8748
Epoch 7/30
7352/7352 [==============================] - 75s 10ms/step - loss: 0.2206 - acc: 0.9240 - val_loss: 0.3845 - val_acc: 0.8792


In [75]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  5      414        67        0                   0   
STANDING                 0      102       429        0                   0   
WALKING                  0        0         0      472                  24   
WALKING_DOWNSTAIRS       0        0         0        7                 408   
WALKING_UPSTAIRS         0        0         0       26                  24   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            5  
STANDING                           1  
WALKING                            0  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 421  


In [76]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 1ms/step


In [77]:
score

[0.3366131105210424, 0.9097387173396675]

### 2 LSTM + 2 CNN with 0.6 Dropout

In [103]:

epochs = 30
batch_size = 32
n_hidden_1 = 64
n_hidden_2 = 64

In [104]:
#https://blog.goodaudience.com/predicting-physical-activity-based-on-smartphone-sensor-data-using-cnn-lstm-9182dd13b6bc
model = Sequential()
model.add(Conv1D(128, (3), input_shape=(X_train.shape[1],X_train.shape[2]), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2),padding='same'))
model.add(Dropout(0.6))

model.add(Conv1D(64,(3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2),padding='same'))
model.add(Dropout(0.6))


# Configuring the parameters
model.add(LSTM(n_hidden_1,kernel_initializer='glorot_uniform',return_sequences=True))
# Adding a dropout layer
# Configuring the parameters
model.add(LSTM(n_hidden_2,kernel_initializer='glorot_uniform',return_sequences=False))
# Adding a dropout layer
model.add(Dropout(0.6))

# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 128, 128)          3584      
_________________________________________________________________
batch_normalization_27 (Batc (None, 128, 128)          512       
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 64, 128)           0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 64, 128)           0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 64, 64)            24640     
_________________________________________________________________
batch_normalization_28 (Batc (None, 64, 64)            256       
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 32, 64)            0         
__________

In [105]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [106]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.5455 - acc: 0.7973 - val_loss: 0.3852 - val_acc: 0.8544
Epoch 2/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.2319 - acc: 0.9226 - val_loss: 0.2402 - val_acc: 0.9189
Epoch 3/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.1730 - acc: 0.9366 - val_loss: 0.2434 - val_acc: 0.9084
Epoch 4/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.1738 - acc: 0.9365 - val_loss: 0.2888 - val_acc: 0.8975
Epoch 5/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.1515 - acc: 0.9406 - val_loss: 0.2445 - val_acc: 0.8958
Epoch 6/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.1608 - acc: 0.9392 - val_loss: 0.3236 - val_acc: 0.9013
Epoch 7/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.1455 - acc: 0.9433 - val_loss: 0.2627 - val_acc: 0.9148
Epoch 8

In [107]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  7      401        82        0                   0   
STANDING                 0       83       449        0                   0   
WALKING                  0        0         0      478                  18   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         2        2         0        1                  23   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 443  


In [108]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [109]:
score

[0.2594191223808506, 0.9256871394638616]

### 3 LSTM + 3 CNN with 0.3 Dropout

In [33]:
from keras.layers.normalization import BatchNormalization


In [119]:

epochs = 30
batch_size = 32
n_hidden_1 = 16
n_hidden_2 = 16
n_hidden_3 = 8

In [120]:
#https://blog.goodaudience.com/predicting-physical-activity-based-on-smartphone-sensor-data-using-cnn-lstm-9182dd13b6bc
model = Sequential()
model.add(Conv1D(128, (3), input_shape=(X_train.shape[1],X_train.shape[2]), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2),padding='same'))
model.add(Dropout(0.3))

model.add(Conv1D(64,(3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2),padding='same'))
model.add(Dropout(0.3))

model.add(Conv1D(32,(3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2),padding='same'))
model.add(Dropout(0.3))

# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True))
# Adding a dropout layer
# Configuring the parameters
model.add(LSTM(n_hidden_2,return_sequences=True))
# Adding a dropout layer
model.add(LSTM(n_hidden_3))
model.add(Dropout(0.3))



# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_41 (Conv1D)           (None, 128, 128)          3584      
_________________________________________________________________
batch_normalization_40 (Batc (None, 128, 128)          512       
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 64, 128)           0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 64, 128)           0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 64, 64)            24640     
_________________________________________________________________
batch_normalization_41 (Batc (None, 64, 64)            256       
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 32, 64)            0         
__________

In [121]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [122]:
from keras.callbacks import ModelCheckpoint


In [123]:
filepath="best_model.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
callbacks_list =[checkpoint]

In [124]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,callbacks=callbacks_list,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.9854 - acc: 0.7237 - val_loss: 0.6418 - val_acc: 0.8412

Epoch 00001: val_acc improved from -inf to 0.84119, saving model to best_model.hdf5
Epoch 2/30
7352/7352 [==============================] - 46s 6ms/step - loss: 0.4653 - acc: 0.8800 - val_loss: 0.3590 - val_acc: 0.8948

Epoch 00002: val_acc improved from 0.84119 to 0.89481, saving model to best_model.hdf5
Epoch 3/30
7352/7352 [==============================] - 44s 6ms/step - loss: 0.3322 - acc: 0.9104 - val_loss: 0.2944 - val_acc: 0.9118

Epoch 00003: val_acc improved from 0.89481 to 0.91177, saving model to best_model.hdf5
Epoch 4/30
7352/7352 [==============================] - 45s 6ms/step - loss: 0.2769 - acc: 0.9178 - val_loss: 0.2603 - val_acc: 0.9128

Epoch 00004: val_acc improved from 0.91177 to 0.91279, saving model to best_model.hdf5
Epoch 5/30
7352/7352 [==============================] - 44s 6ms

In [125]:
# loading the weights of best model
model.load_weights("best_model.hdf5")

In [126]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [127]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  6      399        86        0                   0   
STANDING                 0       66       466        0                   0   
WALKING                  0        0         0      491                   4   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        1                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 470  


In [128]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 10s 3ms/step


In [129]:
score

[0.15064731126539455, 0.9443501866304717]

## Summary

1.Tried single layer LSTM models with different dropout,activations and optimizer.

2.Single layer LSTM model with 0.6 dropout ,sigmoid activation and rmsprop optimizer gives the best score of 0.917

3.Tried dual layer LSTM models with different dropout,activations,kernel initializer and optimizer.

4.Dual layer LSTM model with 0.6 dropout ,softmax activation and adam optimizer gives the best score of 0.926

6.Score's were improved after adding batch normalization layer

7.Tried adding CNN Layer to improve the score.

8.3 CNN layers with 3 LSTM layers model gave the best score of 0.944

9.Compared all the results with pretty library.



In [131]:
from prettytable import PrettyTable

x = PrettyTable(["Model","Layers","Dropout","Kernal Initiaalization","Activation","Optimizer","Score"])

x.add_row(["LSTM","single-64","0.6","-","sigmoid","rmsprop","0.917203936206311"])
x.add_row(["LSTM","single-128","0.5","-","sigmoid","adam","0.903291482863929"])
x.add_row(["LSTM","single-64","0.6","-","softmax","adam","0.906684764166949"])
x.add_row(["LSTM","Dual-16+32","0.6","he_normal","sigmoid","adam","0.903630810994231"])
x.add_row(["LSTM","Dual-32+64","0.6","he_normal","sigmoid","adam","0.903630810994231"])
x.add_row(["LSTM","Dual-32+64","0.5","glorot_uniform","softmax","adam","0.915507295554801"])
x.add_row(["LSTM","Dual-32+64","0.6","glorot_uniform","softmax","adam","0.926365795724465"])
x.add_row(["LSTM","Dual-32+64","0.6","glorot_uniform","softmax","rmsprop","0.913471326772989"])
x.add_row(["CNN+LSTM","1 CNN(16)+2 LSTM(32+64)","0.5","glorot_uniform","sigmoid","adam","0.909738717339667"])
x.add_row(["CNN+LSTM","2 CNN(128+64)+2 LSTM(64+64)","0.6","glorot_uniform","softmax","adam","0.925687139463861"])
x.add_row(["CNN+LSTM","3 CNN(128+64+32)+3 LSTM(16+16+8)","0.3","glorot_uniform","softmax","adam","0.944350186630471"])

print(x)

+----------+----------------------------------+---------+------------------------+------------+-----------+-------------------+
|  Model   |              Layers              | Dropout | Kernal Initiaalization | Activation | Optimizer |       Score       |
+----------+----------------------------------+---------+------------------------+------------+-----------+-------------------+
|   LSTM   |            single-64             |   0.6   |           -            |  sigmoid   |  rmsprop  | 0.917203936206311 |
|   LSTM   |            single-128            |   0.5   |           -            |  sigmoid   |    adam   | 0.903291482863929 |
|   LSTM   |            single-64             |   0.6   |           -            |  softmax   |    adam   | 0.906684764166949 |
|   LSTM   |            Dual-16+32            |   0.6   |       he_normal        |  sigmoid   |    adam   | 0.903630810994231 |
|   LSTM   |            Dual-32+64            |   0.6   |       he_normal        |  sigmoid   |    adam 